# Global M. bovis lineages - fetch data

* Brites paper: https://open-research-europe.ec.europa.eu/articles/1-100/v2
* Joe data: https://www.ncbi.nlm.nih.gov/bioproject/PRJNA523164

In [11]:
import sys,os,shutil,subprocess
import glob,random
from importlib import reload
import numpy as np
import pandas as pd
pd.set_option('display.width', 200)
import pylab as plt
import matplotlib as mpl
from Bio import SeqIO

## Brites data

In [25]:
brites=pd.read_csv('brites_table1.csv')
brites=brites.dropna(subset=['Accession_Number'])
brites.Clonal_Complex.value_counts()
x=brites.Country.value_counts()
countries=list(x[-24:].index)

In [ ]:
brites[brites.Country=='United Kingdom']

In [4]:
brites=brites[brites.Country!='Ireland']
#low quality
omit = ['ERR017796','SRR7131025','SRR7131117','SRR6797355']

## fetch data

In [13]:
def fetch_data(meta,col,path,omit=[]):
    for run in meta[col]:
        if run in omit:
            continue
        files = glob.glob(os.path.join(path,'%s*.fastq*') %run)
        if len(files)>0:
            continue
        cmd = 'fastq-dump --split-files -O %s %s' %(path,run)
        print (run)
        print (cmd)
        try:
            subprocess.check_output(cmd, shell=True)
        except:
            print ('failed')
            continue
        f=os.path.join(path,'%s*.fastq' %run)
        cmd = 'pigz %s' %f
        print (cmd)
        subprocess.check_output(cmd, shell=True)
        

In [ ]:
path = '/storage/btbgenie/brites'
#uk = brites[brites.Country=='United Kingdom']
rare = brites[brites.Country.isin(countries)]
fetch_data(rare,'Accession_Number',path,omit)

In [6]:
files = glob.glob('/storage/btbgenie/mbovis_sra/*.fastq*')
for f in files:
    name=os.path.splitext(os.path.basename(f))[0].split('_')[0]
    #print (name)
    if name not in list(meta.Accession_Number):
        print (name,os.path.basename(f))
        shutil.move(f,os.path.join('/storage/btbgenie/mbovis_sra_other',os.path.basename(f)))

## Joe data

In [6]:
ukmeta = pd.read_csv('/storage/btbgenie/UK/joe_runtable.txt')

path='/storage/btbgenie/UK'
#fetch_data(ukmeta,'Run',path)

In [7]:
#check against brites for dups
ukmeta.Run
brites[brites.Accession_Number.isin(ukmeta.Run)]

,Accession_Number,Clonal_Complex,Country,Gnumber,Host,Map,SIB_Number,Sample.included.in.Loiseau.et.al..2020
